In [21]:
class MyQuoRing(Ring):
    def __init__(self,p_ring,qout):
        self.p_ring=p_ring
        self.n=quot.degree()
        self.quot=quot
    def generator(self):
        for p in self.enumerate_ring():
            if self.is_generator(p):
                return p
    def size(self):
        return self.p_ring.characteristic()**self.n
    def is_generator(self,p):
        factors = [a[0] for a in list(factor(self.size()-1))]
        for f in factors:
            if pow(p,self.size()//f,self.quot)==1:
                return False
        return True
    def elem_prop(self,p):
        if p==1:
            return "order",1
        res=p
        for i in range(2,self.size()+3):
            res=pow(res*p,1,self.quot)
            if res==1:
                return "order",i
            if res==p:
                return "cycle",i
    #does not include 0            
    def enumerate_ring(self):
        return [p for i in range(self.n) for p in self.p_ring.polynomials(of_degree=i)]
    def order_d(self,d):
        possible=[]
        for e in self.enumerate_ring():
            prop,k = self.elem_prop(e)
            if prop=="order" and k==d:
                possible.append(e)
        return sample(possible,1)[0]
        
    #checks if sum_{i=0}^{d-1} a^(ik) = d if k=0 (mod d) else 0
    def vanishing_property(self,a,d):
        p_minus_one=self.p_ring.characteristic()-1
        #print(p_minus_one)
        if p_minus_one%d!=0 or d>p_minus_one:
            return
        print(a,"order",d)
        for k in range(1,2*d+1):
            res = sum(pow(a,i*k,self.quot) for i in range(0,d))
            res = pow(res,1,self.quot)
            if k%d==0:
                if res!=d:
                    return False
            else:
                if res!=0:
                    return False
        return True
    def __str__(self):
        return "My quotient ring of: {} modulo {}".format(self.p_ring,self.quot)

In [34]:
ff=GF(13)
n=2
A=PolynomialRing(ff,['x'])
quot=A("x")^n-4

A_q=MyQuoRing(A,quot)

#[p for p in A.polynomials(of_degree=n)]
elems=A_q.enumerate_ring()

assert len(elems)+1==A_q.size()

props = [(e,A_q.elem_prop(e)) for e in elems]
#print(props)

# for p in props:
#     if p[1][0]=="order":
#         try:
#             A_q.vanishing_property(p[0],p[1][1])
#         except Exception as e:
#             print(e)
A_q.vanishing_property(A("5*x+6"),3)
A_q.order_d(3)

5*x + 6 order 3


6*x + 2

In [138]:
class DFT:
    def __init__(self,quo_ring,d,a=None):
        self.quo_ring=quo_ring
        self.d=d
        
        p_minus_one = self.quo_ring.p_ring.characteristic()-1
        assert p_minus_one%self.d==0 and self.d<=p_minus_one, "{} does not divide {}".format(d,p_minus_one)
        
        self.d_inverse=p_minus_one//self.d
        
        if a==None:
            self.dth_root=self.quo_ring.order_d(d)
        else:
            self.dth_root=a
        if not self.dth_root:
            raise Exception("No element with order {} can be found in: {}".format(self.d,self.quo_ring))
        
        self.dth_root_inverse=pow(self.dth_root,d-1,self.quo_ring.quot)
        assert pow(self.dth_root_inverse*self.dth_root,1,self.quo_ring.quot)==1
        
    def getMat(self,inverse=False):
        if not inverse:
            return matrix([[pow(self.dth_root,(i*j),self.quo_ring.quot) for i in range(self.d)] for j in range(self.d)])
        else:
            return -1*self.d_inverse*matrix([[self.dth_root_inverse^(i*j) for i in range(self.d)] for j in range(self.d)])
    
    def __str__(self):
        return "DFT: \n  d:{}\n  dth_root:{}".format(self.d,self.dth_root)
    
    def check_inverse(self,a):
        a_prime = a*self.getMat()
        print(a_prime)
        a_rec = a_prime*self.getMat(inverse=True)
        a_rec=[pow(a_rec[0][i],1,self.quo_ring.quot) for i in range(self.d)]
        print(a_rec)
        assert a == a_rec, f"check inverse failed {a} != {a_rec}"

In [139]:
d=3
a_vec=sample(elems,d)
a_vec=[A("x+2"),A("x+3"),A("x+4")]

In [140]:
dft=DFT(A_q,d,a=A("5x+6"))
#dft=DFT(A_q,d)
print(dft)
print(dft.getMat())
print(dft.getMat(inverse=True))
print(a_vec)
a_rec = dft.check_inverse(matrix(a_vec))

DFT: 
  d:3
  dth_root:5*x + 6
[      1       1       1]
[      1 5*x + 6 8*x + 6]
[      1 8*x + 6 5*x + 6]
[                            9                             9                             9]
[                            9                       7*x + 2              4*x^2 + 6*x + 12]
[                            9              4*x^2 + 6*x + 12 9*x^4 + x^3 + 6*x^2 + 3*x + 3]
[x + 2, x + 3, x + 4]
[3*x + 9 8*x + 5 5*x + 5]
[x + 2, x + 3, x + 4]


AssertionError: check inverse failed [x + 2 x + 3 x + 4] != [x + 2, x + 3, x + 4]

In [6]:
sample(elems,4)

[7*x + 6, 8*x + 6, 12*x + 10, 6*x]

In [126]:
a=A("5x+6")
for i in range(4):
    print(a**i,pow(a,i,A("x^2-4")))
    

1 1
5*x + 6 5*x + 6
12*x^2 + 8*x + 10 8*x + 6
8*x^3 + 8*x^2 + 7*x + 8 1


In [130]:
A("5x+6")*9

6*x + 2